## Load data

In [ ]:
from PIL import Image
from numpy import asarray
def transformImage(filename):
    image = Image.open(filename)
    # convert image to numpy array
    data = asarray(image)
    return data

inputs = []
for img in range(18,25):
    filename = str(img)+'.png'
    data = transformImage(filename)
    pixels = []
    means = []
    for i in range(0,len(data)):
        localMean = 0.0
        for j in range(0,len(data[i])):
            mean = 0.0
            for d in range(0,4):
                mean += data[i][j][d]
            mean /= 4
            localMean += mean
        localMean /= 128
        means.append(localMean)
    inputs.append(means) 
for img in range(1,17):
    filename = str(img)+'.png'
    data = transformImage(filename)
    pixels = []
    means = []
    for i in range(0,len(data)):
        localMean = 0.0
        for j in range(0,len(data[i])):
            mean = 0.0
            for d in range(0,4):
                mean += data[i][j][d]
            mean /= 4
            localMean += mean
        localMean /= 128
        means.append(localMean)
    inputs.append(means)
    
outputs=[]
outputNames = ['0','1']

### plot data on classes 

In [ ]:


import matplotlib.pyplot as plt
labels = set(outputs)
labelsNames = []
noData = len(inputs)
print(noData)

inputs = inputs[1:]
for label in labels:
    x=[]
    y=[]
    for dt in range(len(inputs)):
        print(inputs[dt])
        if outputs[dt] == label:
            x.append((inputs[dt][0]+inputs[dt][1])/2)
            y.append((inputs[dt][2]+inputs[dt][3])/2)
    plt.scatter(x, y, label = label)

plt.xlabel('mean sepal')
plt.ylabel('mean petal')
plt.legend()
plt.show()


### separating training data from test data 20%-80%

In [ ]:
import numpy as np 
import math

np.random.seed(5)

# generate the positions of the data that will be a part of the test data
noTestDataIndexes = math.floor( 0.2 * len(inputs))
testDataIndexes = []
for index in range(0,noTestDataIndexes):
    testDataIndexes.append(np.random.randint(0,len(inputs)))

inputTest = []
outputTest = []
inputTraining = []
outputTraining = []
for i in range (0,len(inputs)):
    if i in testDataIndexes :
        inputTest.append(inputs[i])
        outputTest.append(outputs[i])
    else:
        inputTraining.append(inputs[i])
        outputTraining.append(outputs[i])

**normalize data**

In [ ]:
from sklearn.preprocessing import StandardScaler

def normalisation(trainData, testData):
    scaler = StandardScaler()
    if not isinstance(trainData[0], list):
        #encode each sample into a list
        trainData = [[d] for d in trainData]
        testData = [[d] for d in testData]
        
        scaler.fit(trainData)  #  fit only on training data
        normalisedTrainData = scaler.transform(trainData) # apply same transformation to train data
        normalisedTestData = scaler.transform(testData)  # apply same transformation to test data
        
        #decode from list to raw values
        normalisedTrainData = [el[0] for el in normalisedTrainData]
        normalisedTestData = [el[0] for el in normalisedTestData]
    else:
        scaler.fit(trainData)  #  fit only on training data
        normalisedTrainData = scaler.transform(trainData) # apply same transformation to train data
        normalisedTestData = scaler.transform(testData)  # apply same transformation to test data
    return normalisedTrainData, normalisedTestData


normalizedTrainInput, normalizedTestInput = normalisation(inputTraining,inputTest)
print(normalizedTrainInput)
print(normalizedTestInput)

In [ ]:
plt.hist(outputTraining, rwidth = 0.8)
plt.xticks(np.arange(len(outputNames)), outputNames)
plt.show()


    **ANN Classification (logic)**

In [ ]:
class Neuron:
    def __init__(self,w=[],out=None,delta=0.0):
        self.weights = w
        self.output = out
        self.delta = delta
    
def initialisation(inputNumberNeurons,outputNumberNeurons,hiddenNeuronsNumber):
    networkStruct = []
    hiddenLayer = []
    # construct the hidden layer  
    for neuron in range(0,hiddenNeuronsNumber):
        # each neuron starts with random weights 
        neuronWeights = []
        for weight in range(0,inputNumberNeurons+1):
            neuronWeights.append(random())
        neuronToAdd = Neuron(neuronWeights)
        # add each neuron to the hidden layer
        hiddenLayer.append(neuronToAdd)
    # add the hidden layer to the structure of the network
    networkStruct.append(hiddenLayer)
    
    outputLayer = []        
    # construct the output layer
    for neuron in range(0,outputNumberNeurons):
        # each neuron starts with random weights 
        neuronWeights = []
        for weight in range(0,hiddenNeuronsNumber+1):
            neuronWeights.append(random())
        neuronToAdd =  Neuron(neuronWeights)
        # add each neuron to the output layer
        outputLayer.append(neuronToAdd)
    # add the output layer to the structure of the network
    networkStruct.append(outputLayer)
    return networkStruct

        
def activateNeuron(inputValues,weights):
    res = 0.0
    for i in range(0,len(inputValues)):
        res += inputValues[i] * weights[i]
    # adding w0
    res += weights[len(inputValues)]
    return res

def activationFunction(info):
    return 1/(1+math.exp(-info))

def forwardPropagationInformation(network , inputValues):
    # each layer compute the information through the activation of every neuron of the layer
    for layer in network:
        computedInputs = []
        for neuron in layer : 
            # gather the information for one neuron and compute the information based on the weights
            getInformationForNeuron = activateNeuron(inputValues,neuron.weights)
            # apply activation function
            neuron.output  = activationFunction(getInformationForNeuron)
            computedInputs.append(neuron.output)
        # transfer the outputs from the k-layer to the (k+1) layer as input
        inputValues = computedInputs
    # the computedInputs of the last layer is the output
    return inputValues


def derivativeSigmoid(outputValue):
    return  outputValue * ( 1 - outputValue)

def backwardPropagationError(network , expectedValues):
    # the error is propagated from the last layer of the netwrok to the first
    for i in range( len(network)-1,-1,-1):
        currentLayer = network[i]
        errors = []
        # calculate the errors from the output layer
        # for each neuron as the error that propagates back
        # has to be proportional with the information that came into the neuron
        if i == len(network) - 1:
            # this is the output layer
            for j in range(0,len(currentLayer)):
                # iterate through the neurons of the layer
                neuron =  currentLayer[j]
                # the error is calculated as the difference between the expected value and 
                # the value of the output layer
                errors.append( expectedValues[j] - neuron.output)
        else:
            # this is an internal layer
            # for each neuron we calculate the error gathered from 
            # the neuron fron the (k+1) layer
            for j in range(0,len(currentLayer)):
                errorValue = 0.0
                nextLayer = network[i+1]
                for neuron in nextLayer:
                    # get the value of the error for each of the next layers' neurons
                    errorValue += neuron.weights[j] * neuron.delta
                errors.append(errorValue)
        #update delta for each neuron of the current layer     
        for j in range(0,len(currentLayer)):
            currentLayer[j].delta = errors[j] * derivativeSigmoid(currentLayer[j].output)
            

def updateWeights(network,sample,learningRate):
    # update weights for each neuron of the network
    for i in range(0,len(network)):
        inputValues = sample[:-1]
        if i > 0:
            # for the layers of the network 
            # except the input layer the values are the computed ones from the neurons
            inputValues = []
            for neuron in network[i]:
                inputValues.append(neuron.output)
        for neuron in network[i]:
            for j in range(len(inputValues)):
                neuron.weights[j] += learningRate * neuron.delta * inputValues[j]
            neuron.weights[-1] += learningRate * neuron.delta 

MAX = 10
def sparseCategorialCrossentropy(realLabels,computedOutputs):
        labelNames = list(set(realLabels))
        labelNames.sort()
        realClasses=[]
        for c in realLabels:
            realClasses.append(labelNames.index(c)+1)
        predictedClasses=[]
        for p in computedOutputs:
            max = p[0]
            index = 1 
            if p[1] > max:
                index = 2
                max = p[1]
            if p[2] > max:
                index = 3
            predictedClasses.append(index)
        sparseCatCrossentropy=0
        nr=0
        maxVal = -9999999
    
        for i in range(0,len(realClasses)):
                if predictedClasses[i] == 1:
                    nr += 1
                    sparseCatCrossentropy += MAX
                else:
                    val = realClasses[i]*math.log(predictedClasses[i]) + (1-realClasses[i])*math.log(abs(1-predictedClasses[i]))
                    sparseCatCrossentropy += val
                    if val > maxVal:
                        maxVal = val
        sparseCatCrossentropy = sparseCatCrossentropy - MAX*nr + maxVal
        return -1/len(realClasses)*sparseCatCrossentropy
        
def training(network,data,outputs,numberClasses,learningRate,noEpochs):
    
    for epoch in range(0,noEpochs):
        print('Epoch '+str(epoch)+'loss=')
        overAllError = 0.0
        nrOut = 0 
        lossComputedValues = []
        for sample in data :
            inputValues = sample
            computedValues = forwardPropagationInformation(network,inputValues)
            lossComputedValues.append(computedValues)
            expected = [ 0 for _ in range(numberClasses)]
            expected[outputs[nrOut]] = 1
            computedLabels = [0 for _ in range(numberClasses)]
            computedLabels[computedValues.index(max(computedValues))] = 1
            computedValues = computedLabels
            currentError = sum([(expected[i] - computedValues[i]) ** 2 for i in range(0, len(expected))])
            overAllError += currentError
            backwardPropagationError(network,expected)
            updateWeights(network,sample,learningRate)
            nrOut += 1
        print(sparseCategorialCrossentropy(outputs,lossComputedValues))
        
def eval(network,data,numberClasses):
    computedOutputs = []
    for dt in data:
        print(dt)
        computedOutput = forwardPropagationInformation(network,dt[:-1])
        computedLabels = [0 for i in range(numberClasses)]
        computedLabels[computedOutput.index(max(computedOutput))] = 1
        computedOutput = computedLabels
        computedOutputs.append(computedOutput.index(max(computedOutput)))
    return computedOutputs
        
        
def evalMultiClass(realLabels, computedLabels, labelNames):
    from sklearn.metrics import confusion_matrix

    confMatrix = confusion_matrix(realLabels, computedLabels)
    acc = sum([confMatrix[i][i] for i in range(len(labelNames))]) / len(realLabels)
    precision = {}
    recall = {}
    for i in range(len(labelNames)):
        precision[labelNames[i]] = confMatrix[i][i] / sum([confMatrix[j][i] for j in range(len(labelNames))])
        recall[labelNames[i]] = confMatrix[i][i] / sum([confMatrix[i][j] for j in range(len(labelNames))])
    return acc, precision, recall, confMatrix
            
        
            
            

### leran model

In [ ]:

        


numberClasses = len(set(outputs))
numberInputNeurons = len(normalizedTrainInput)
network = initialisation(numberInputNeurons,numberClasses,10)
training(network,normalizedTrainInput,outputTraining,numberClasses,0.1,100)
computedOutputs =  eval(network,normalizedTrainInput,numberClasses)


## predict classes for test data

In [ ]:
training(network,normalizedTestInput,outputTest,numberClasses,0.1,100)
computedOutputs =  eval(network,normalizedTestInput,numberClasses)

### calculate error



In [ ]:
error = 0.0
for guess, real in zip(computedOutputs, outputTest):
    if guess != real:
        error += 1
error = error / len(outputTest)
print("classification error (manual) with code: ", error)


from sklearn.metrics import accuracy_score
error = 1 - accuracy_score(outputTest, computedOutputs)
print("classification error (tool) with code: ", error)


In [ ]:
outputNames = ['0','1','2','3','4','5','6','7','8','9']
acc, precision, recall,matrix = evalMultiClass(np.array(outputTest), computedOutputs, outputNames)
print('acc: ', acc)
print('precision: ', precision)
print('recall: ', recall)
